In [1]:
import ROOT

from resolution import *
from digitizer import *
from pile_up import *

from tqdm import tqdm

runs = [ "19F_pg", "19F_pa" ]

keV = 1e3

a = 15
energyCut1 = 0.1 * keV
energyCut2 = 0.1 * keV
energyCut3 = 0.1 * keV
energyCut4 = 0.1 * keV
energyCut5 = 0.1 * keV
energyCut6 = 0.1 * keV

probPU = 0.001
energyBGOsumPU = 0

rand = ROOT.TRandom3( )

Welcome to JupyROOT 6.28/04


In [2]:
for run in runs:

    inFile = ROOT.TFile( "output/{}.root".format(run) )

    tree = inFile.Get( "EdepBGO" )

    h1 = ROOT.TH1D( "BGO1","BGO1", 9000, 0, 18000 )
    h2 = ROOT.TH1D( "BGO2","BGO2", 9000, 0, 18000 )
    h3 = ROOT.TH1D( "BGO3","BGO3", 9000, 0, 18000 )
    h4 = ROOT.TH1D( "BGO4","BGO4", 9000, 0, 18000 )
    h5 = ROOT.TH1D( "BGO5","BGO5", 9000, 0, 18000 )
    h6 = ROOT.TH1D( "BGO6","BGO6", 9000, 0, 18000 )
    h7 = ROOT.TH1D( "BGOsum","BGOsum", 9000, 0, 18000 )

    for entryNum in tqdm( range( tree.GetEntries( ) ) ):
        
        tree.GetEntry( entryNum )
        
        energyBGO1 = getattr( tree, "BGO1" ) * keV
        energyBGO2 = getattr( tree, "BGO2" ) * keV
        energyBGO3 = getattr( tree, "BGO3" ) * keV
        energyBGO4 = getattr( tree, "BGO4" ) * keV
        energyBGO5 = getattr( tree, "BGO5" ) * keV
        energyBGO6 = getattr( tree, "BGO6" ) * keV

        if( energyBGO1 > 0 ): energyBGO1 = rand.Gaus( energyBGO1, resolution( energyBGO1, "1" ) )
        if( energyBGO2 > 0 ): energyBGO2 = rand.Gaus( energyBGO2, resolution( energyBGO2, "2" ) )
        if( energyBGO3 > 0 ): energyBGO3 = rand.Gaus( energyBGO3, resolution( energyBGO3, "3" ) )
        if( energyBGO4 > 0 ): energyBGO4 = rand.Gaus( energyBGO4, resolution( energyBGO4, "4" ) )
        if( energyBGO5 > 0 ): energyBGO5 = rand.Gaus( energyBGO5, resolution( energyBGO5, "5" ) )
        if( energyBGO6 > 0 ): energyBGO6 = rand.Gaus( energyBGO6, resolution( energyBGO6, "6" ) )

        if( trigger( energyBGO1, energyCut1, a ) ): energyBGO1 = 0
        if( trigger( energyBGO2, energyCut2, a ) ): energyBGO2 = 0
        if( trigger( energyBGO3, energyCut3, a ) ): energyBGO3 = 0
        if( trigger( energyBGO4, energyCut4, a ) ): energyBGO4 = 0
        if( trigger( energyBGO5, energyCut5, a ) ): energyBGO5 = 0
        if( trigger( energyBGO6, energyCut6, a ) ): energyBGO6 = 0

        energyBGOsum = energyBGO1 + energyBGO2 + energyBGO3 + energyBGO4 + energyBGO5 + energyBGO6

        if( energyBGO1 != 0 ): h1.Fill( energyBGO1 )
        if( energyBGO2 != 0 ): h2.Fill( energyBGO2 )
        if( energyBGO3 != 0 ): h3.Fill( energyBGO3 )
        if( energyBGO4 != 0 ): h4.Fill( energyBGO4 )
        if( energyBGO5 != 0 ): h5.Fill( energyBGO5 )
        if( energyBGO6 != 0 ): h6.Fill( energyBGO6 )
    
        if( energyBGOsum != 0 ):
            if( pile_up( probPU ) ): energyBGOsumPU += energyBGOsum
            else: 
                h7.Fill( energyBGOsum + energyBGOsumPU )
                energyBGOsumPU = 0

    outHistFile = ROOT.TFile.Open( "histograms/{}.root".format(run), "RECREATE" )

    outHistFile.cd( )

    h1.Write( )
    h2.Write( )
    h3.Write( )
    h4.Write( )
    h5.Write( )
    h6.Write( )
    h7.Write( )

    outHistFile.Close( )

    inFile.Close( )

100%|██████████| 1000000/1000000 [00:27<00:00, 36030.95it/s]
